In [2]:
    
import os
import re
import math
import random
import warnings

from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import gensim
from nltk.tokenize import word_tokenize
import dateutil.parser


%matplotlib inline


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\glin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Read all files:

qs = pd.read_csv('PostQuestionsFiltered_V4_parsed.tsv',delimiter='\t',encoding='utf-8')
comments = pd.read_csv('CommentsFiltered_v3.tsv',delimiter='\t',encoding='utf-8')
tags = pd.read_csv('tags.csv')
answers = pd.read_csv('PostAnswersFiltered_V4_parsed.tsv',delimiter='\t',encoding='utf-8')


C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Build out Recommender - Find Similar Questions

In [86]:
qs.head()

,Unnamed: 0,Unnamed: 0.1,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,...,seaborn,geospatial,stata,plyr,pie-chart,graphviz,spss,diagram,qlikview,altair
0,0,0,7844124,Disable map controls in Google Maps embed,<p>Is it possible to disable map controls by adding some variables to the Google Maps embed code?</p>,7846901.00,5,1,NaN,2011-10-21 01:35:21.823000+00:00,...,0,0,0,0,0,0,0,0,0,0
1,1,1,8018288,Stack Points in ggplot,"<p>I am making a dotplot using <code>ggplot</code> with the code and data that is below which produces the following the graph.</p>\r\r\n\r\r\n<pre><code>ggplot(data=holder, aes(x=Coef, y=CoefShort, colour=factor(Name))) + geom_point() + labs(x=""Value"", y=""Coefficient"") + scale_colour_discrete(""Model"")\r\r\n</code></pre>\r\r\n\r\r\n<p><img src=""https://i.stack.imgur.com/G6wng.png"" alt=""dotplot""></p>\r\r\n\r\r\n<p>Their is a significant amount of overplotting and I would like to create some vertical seperation between dots. But it has to be systematic, i.e. the order is always red-green-blue.</p>\r\r\n\r\r\n<p><code>geom_stack</code> only seems to work when the x value is the same, not when the y value is the same. I tried using the opposite axes (swapping the x and y), doing a <code>position_dodge()</code> then doing a <code>coord_flip()</code>. This only works with bars, as seen below.</p>\r\r\n\r\r\n<pre><code>ggplot(data=holder, aes(y=Coef, x=CoefShort, fill=factor(Name))) + geom_bar(position=""dodge"", aes(ymax=Coef)) + labs(y=""Value"", x=""Coefficient"") + scale_fill_discrete(""Model"") + coord_flip()\r\r\n</code></pre>\r\r\n\r\r\n<p><img src=""https://i.stack.imgur.com/Xe1il.png"" alt=""Vertically dodged bar chart""></p>\r\r\n\r\r\n<p>Any ideas how to achieve that stacking with <code>geom_point()</code>? Thanks.</p>\r\r\n\r\r\n<p>The data:</p>\r\r\n\r\r\n<p><code>structure(list(Coef = c(-3875.46969970703, 7871.08213392282, \r\r\n1120.33185255098, 1510.13540851347, 1439.07714113149, 1800.92398445336, \r\r\n-3760.05411752962, 8183.74295221482, 1126.98290537184, 1517.99524139857, \r\r\n1442.73063836897, 1808.03721179571, -90.6507661872817, -71.7225864185226, \r\r\n-103.615416254984, -732.167583256825, -1075.67574987664, -1908.56266462926, \r\r\n-7362.80215630299, 8886.12888250011, 655.767448263926, 848.716877683527, \r\r\n869.395903077767, 998.254438325812, -211.682481369473, -303.310032581644, \r\r\n-506.1995360406, -978.697664841985, -1440.30190190734, -2325.22236024601, \r\r\n2625.94998656519, 3573.68798735398, 4217.82910198788, 4534.8789695778, \r\r\n4967.19941000705, 5072.02764498623, 5419.64684461491), Name = c(1, \r\r\n1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, \r\r\n3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3), CoefShort = structure(c(1L, \r\r\n2L, 3L, 4L, 5L, 6L, 1L, 2L, 3L, 4L, 5L, 6L, 7L, 8L, 9L, 10L, \r\r\n11L, 12L, 1L, 2L, 3L, 4L, 5L, 6L, 7L, 8L, 9L, 10L, 11L, 12L, \r\r\n13L, 14L, 15L, 16L, 17L, 18L, 19L), .Label = c(""(Intercept)"", \r\r\n""carat"", ""Good"", ""Very Good"", ""Premium"", ""Ideal"", ""E"", ""F"", ""G"", \r\r\n""H"", ""I"", ""J"", ""SI2"", ""SI1"", ""VS2"", ""VS1"", ""VVS2"", ""VVS1"", ""IF""\r\r\n), class = ""factor"")), .Names = c(""Coef"", ""Name"", ""CoefShort""\r\r\n), class = ""data.frame"", row.names = c(NA, -37L))</code></p>",8018341.00,1,0,NaN,2011-11-05 05:01:34.233000+00:00,...,0,0,0,0,0,0,0,0,0,0
2,2,2,8227828,RGB to HSB Algorithm,"<blockquote>\r\r\n <p><strong>Possible Duplicate:</strong><br>\r\r\n <a href=""https://stackoverflow.com/questions/3018313/algorithm-to-convert-rgb-to-hsv-and-hsv-to-rgb"">Algorithm to convert RGB to HSV and HSV to RGB?</a> </p>\r\r\n</blockquote>\r\r\n\r\r\n\r\r\n\r\r\n<p>I'm trying to find an algorithm (prefferabley represented in C++) that converts from RGBto HSB values so that I can process it accordingly. Anyone got any suggestions?</p>",8227891.00,2,1,NaN,2011-11-22 13:45:51.707000+00:00,...,0,0,0,0,0,0,0,0,0,0
3,3,3,7976438,I have a few EMF files. How I c

In [13]:
row_iterator = qs.iterrows()
keywords = [
   'javascript', 'java', 'c#', 'php', 'python', 'c++',
       'node.js', 'objective-c', 'vb.net', 'scala', 'matlab', 'perl', 'delphi',
       'matplotlib', 'animation', 'd3', 'ggplot2', 'plot', 'graph', 'chart',
       'highcharts', 'vbscript', 'colors', 'pyspark', 'dplyr', 'f#', '3d',
       'sas', 'fortran', 'maps', 'lisp', 'julia', 'powerbi', 'drawing', 'line',
       'plotly', 'bar-chart', 'visualization', 'tableau', 'seaborn',
       'geospatial', 'stata', 'plyr', 'pie-chart', 'graphviz', 'spss',
       'diagram', 'qlikview', 'altair'
]
row_keywords = []
for row in row_iterator:
    single_row_keywords = [keyword for keyword in keywords if row[1][keyword] == 1]
    single_row_keystring = " ".join(single_row_keywords)
    row_keywords.append(single_row_keystring)
row_keywords[0:10]

['plot',
 'graph',
 'c++ graph',
 'c# chart',
 'line',
 'ggplot2 plot',
 'graph',
 'chart',
 'graph',
 'visualization']

In [14]:
np_array_of_row_keywords = np.array(row_keywords)
qs["new_tags"] = np_array_of_row_keywords

In [15]:
qs.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,...,plyr,pie-chart,graphviz,spss,diagram,qlikview,altair,r,parsedtags,new_tags
0,15537402,using command on a gnplot script,<p>Im using a script on a mac bash shell that ...,NaN,1,2,NaN,2013-03-21 00:34:11.173000+00:00,NaN,2013-03-21 00:44:12.973000+00:00,...,0,0,0,0,0,0,0,0,plot|,plot
1,15856146,Applying Orthographic projection or frustum ef...,<p>I know that normalised coordinates should b...,15858157.0,1,0,NaN,2013-04-06 21:21:10.723000+00:00,NaN,2013-04-07 02:13:30.390000+00:00,...,0,0,0,0,0,0,0,0,graph|,graph
2,15428854,How to implement both scalar and vector additi...,"<p>I'm working on a Vector2D class, and I thin...",15429296.0,3,0,NaN,2013-03-15 09:33:56.357000+00:00,NaN,2013-03-15 10:20:14.973000+00:00,...,0,0,0,0,0,0,0,0,c++|graph|,c++ graph
3,15517350,how to increase speed of tchart refresh()?,<p>I have 16 graphs[maximum ] with 4 fastlines...,15526751.0,1,0,NaN,2013-03-20 07:07:09.313000+00:00,0.0,2013-06-10 13:13:10.997000+00:00,...,0,0,0,0,0,0,0,0,c#|chart|,c# chart
4,15445313,What does $ROOT mean in a Mac Terminal?,<p>I received some command line instructions:<...,15445328.0,2,0,NaN,2013-03-16 03:45:52.347000+00:00,NaN,2013-03-16 03:50:07.557000+00:00,...,0,0,0,0,0,0,0,0,line|,line


In [16]:
combined = pd.merge(qs, answers, how='left', left_on = 'accepted_answer_id', right_on = 'id')

In [17]:
combined.to_csv('combined.csv')

In [27]:
combined.head()

,id_x,title_x,body_x,accepted_answer_id_x,answer_count_x,comment_count_x,community_owned_date_x,creation_date_x,favorite_count_x,last_activity_date_x,...,last_editor_display_name_y,last_editor_user_id_y,owner_display_name_y,owner_user_id_y,parent_id_y,post_type_id_y,score_y,tags_y,view_count_y,cleaned_body
0,15537402,using command on a gnplot script,<p>Im using a script on a mac bash shell that ...,NaN,1,2,NaN,2013-03-21 00:34:11.173000+00:00,NaN,2013-03-21 00:44:12.973000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15856146,Applying Orthographic projection or frustum ef...,<p>I know that normalised coordinates should b...,15858157.0,1,0,NaN,2013-04-06 21:21:10.723000+00:00,NaN,2013-04-07 02:13:30.390000+00:00,...,NaN,NaN,NaN,3758484.0,15856146.0,2.0,0.0,NaN,NaN,To my eyes it appears correct. If your screen...
2,15428854,How to implement both scalar and vector additi...,"<p>I'm working on a Vector2D class, and I thin...",15429296.0,3,0,NaN,2013-03-15 09:33:56.357000+00:00,NaN,2013-03-15 10:20:14.973000+00:00,...,NaN,649665.0,NaN,649665.0,15428854.0,2.0,1.0,NaN,NaN,It's not clear what you're trying to do. The...
3,15517350,how to increase speed of tchart refresh()?,<p>I have 16 graphs[maximum ] with 4 fastlines...,15526751.0,1,0,NaN,2013-03-20 07:07:09.313000+00:00,0.0,2013-06-10 13:13:10.997000+00:00,...,NaN,NaN,NaN,1309861.0,15517350.0,2.0,2.0,NaN,NaN,"I have made a simple code, where I have added..."
4,15445313,What does $ROOT mean in a Mac Terminal?,<p>I received some command line instructions:<...,15445328.0,2,0,NaN,2013-03-16 03:45:52.347000+00:00,NaN,2013-03-16 03:50:07.557000+00:00,...,NaN,NaN,NaN,14860.0,15445313.0,2.0,1.0,NaN,NaN,"is an environment variable, plain and simple...."


In [75]:
qs.to_csv('new_qs.csv')

### TF-IDF Recommend Questions

In [18]:

# questions

raw_documents= qs['title'] + qs['new_tags']
raw_documents = raw_documents
qs = qs

print("Number of Questions:",len(raw_documents))

    #Tokenizing data
gen_docs = [[w.lower() for w in word_tokenize(text)] 
                for text in raw_documents]

    # Create dictionary
dictionary = gensim.corpora.Dictionary(gen_docs)

    # Creat Document-Term Matrix
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

    # Creat TF-IDF Model
tf_idf = gensim.models.TfidfModel(corpus)

    # Creat Similarity Checker
similar_qs = gensim.similarities.Similarity("",tf_idf[corpus],num_features=len(dictionary))

print("Processing Completed!")


Number of Questions: 429665


C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-

C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-

Processing Completed!


In [25]:
class tfModel():
    def __init__(self, question_file):
        qs = pd.read_csv(question_file)
        qs = qs.fillna(value={'new_tags': ''})
        raw_documents = qs['title'] + qs['new_tags']
        # sample to smaller number of documents
        raw_documents = raw_documents
        qs = qs
        self.questions = qs
        tokenized_docs = [[w.lower() for w in word_tokenize(text)] 
                            for text in raw_documents]
        self.dictionary = gensim.corpora.Dictionary(tokenized_docs)
        corpus = [self.dictionary.doc2bow(tokenized_doc) for tokenized_doc in tokenized_docs]
        tf_idf = gensim.models.TfidfModel(corpus)
        self.tf_idf = tf_idf
        self.similarity_checker = gensim.similarities.Similarity("",self.tf_idf[corpus],num_features=len(self.dictionary))
    
    def get_similar_documents(self, query, num_results=5, threshold=0.10):
        tokenized_query = [w.lower() for w in word_tokenize(query)]
        query_bag_of_words = self.dictionary.doc2bow(tokenized_query)
        query_tf_idf = self.tf_idf[query_bag_of_words]
        question_similarities = self.similarity_checker[query_tf_idf]
        print("Q Similarities", len(question_similarities), question_similarities)

        # Display similar questions from the past:

        questions_copy = self.questions.copy()
        questions_copy['similarity'] = question_similarities
        questions_above_threshold_similarity = questions_copy[questions_copy['similarity'] >= threshold]
        questions_above_threshold_similarity = questions_above_threshold_similarity.sort_values('similarity',ascending=False)
        
        return questions_above_threshold_similarity['title'].head(num_results)

def get_document_similarities_for_query(query):
    query_doc = [w.lower() for w in word_tokenize(query)]
    query_doc_bow = dictionary.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    q_sim=similar_qs[query_doc_tf_idf]
    sim_threshold=0.10

    # Display similar questions from the past:

    qs['Similarity']=q_sim
    ques=qs[qs['Similarity']>=sim_threshold]
    ques=qs.sort_values('Similarity',ascending=False)
        
    result = ques['title'].head()
    results.append(input_query)
    results.append(result)

def similar_ques(query_list):
    results = []
    for input_query in query_list:
        query_doc = [w.lower() for w in word_tokenize(input_query)]
        query_doc_bow = dictionary.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf[query_doc_bow]
        q_sim=similar_qs[query_doc_tf_idf]
        sim_threshold=0.10

        # Display similar questions from the past:

        qs['Similarity']=q_sim
        ques=qs[qs['Similarity']>=sim_threshold]
        ques=qs.sort_values('Similarity',ascending=False)
        
        result = ques['title'].head()
#         print(result)
        results.append(input_query)
        results.append(result)
    
    return results

In [20]:
import pickle
with open('tf_idf_model.p', 'wb') as model_file:
    pickle.dump(tfModel, model_file)

In [ ]:
m = tfModel()

In [ ]:
get_similar_documents('Is there a way to visualize the distribution of my data?')

In [25]:
results = similar_ques(Query_List)

C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
results

['Is there a way to visualize the distribution of my data?',
 6034                      How to visualize graphson data?
 7115                How can I visualize categorical data?
 3660                 Normal distribution in linear models
 9127    Is there a way to import a 3D model into Android?
 7712    How can i use the data from Hbase to visualize...
 Name: title, dtype: object,
 'How do I show data on a map?',
 6051                  How to show only labels with data?
 8936                Googlemaps' map don't show on mobile
 3516    How do I get more than one map on a single page?
 4104           How do I update code to show random walk?
 1237          markers doesn't show on the google map API
 Name: title, dtype: object,
 'How can I illustrate changes in my data over time?',
 6489    Why dont all my functions apply changes to my ...
 7897    How can I keep a d3 mouseover open while my mo...
 8839    How can i change the speed on my Timer() durin...
 1611             How to spread

### Word2Vec Recommendation

In [71]:
# import text2vec
import spacy

In [64]:
# import imp
# imp.reload(text2vec)

<module 'text2vec' from 'C:\\Users\\glin\\Desktop\\YQ\\Fall Capstone\\Final data\\text2vec.py'>

In [56]:
# import en_core_web_sm

In [72]:
import spacy

In [73]:
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [47]:
doc_list = raw_documents[:1000]

In [65]:
# input: List of Documents, doc_list is a list of documents/paragraphs/sentences.

t2v = text2vec.text2vec(doc_list)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [30]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [31]:
# Download dataset

# dataset = api.load("text8")
# data = [d for d in dataset]

# # Split the data into 2 parts. Part 2 will be used later to update the model
# data_part1 = data[:1000]
# data_part2 = data[1000:]

# Train Word2Vec model. Defaults result vector size = 100
model = Word2Vec(raw_documents, min_count = 0, workers=cpu_count())

# Get the word vector for given word
# model['topic']
#> array([ 0.0512,  0.2555,  0.9393, ... ,-0.5669,  0.6737], dtype=float32)

model.most_similar('topic')
#> [('discussion', 0.7590423822402954),
#>  ('consensus', 0.7253159284591675),
#>  ('discussions', 0.7252693176269531),
#>  ('interpretation', 0.7196053266525269),
#>  ('viewpoint', 0.7053568959236145),
#>  ('speculation', 0.7021505832672119),
#>  ('discourse', 0.7001898884773254),
#>  ('opinions', 0.6993060111999512),
#>  ('focus', 0.6959210634231567),
#>  ('scholarly', 0.6884037256240845)]

# # Save and Load Model
# model.save('newmodel')
# model = Word2Vec.load('newmodel')

C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


KeyError: "word 'topic' not in vocabulary"

In [35]:
model.save('newmodel')
Word2VecModel = Word2Vec.load('newmodel')

C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [37]:
print(Word2VecModel)

Word2Vec(vocab=112, size=100, alpha=0.025)


In [38]:
# train word2vec model
new_model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [39]:
train = raw_documents

In [40]:
new_model.build_vocab(train, progress_per=200)

new_model.train(train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

KeyboardInterrupt: 

In [ ]:

def similar_ques_word2vec(v, n = 6):
    
    # extract most similar products for the input vector
    ms = new_model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [36]:
    # Creat Word2Vec Model
# tf_idf = gensim.models.Word2VecModel(corpus)

    # Creat Similarity Checker
word_similar_qs = gensim.similarities.Similarity("",Word2VecModel[corpus],num_features=len(dictionary))

C:\Users\glin\AppData\Local\Continuum\anaconda3.2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


TypeError: unhashable type: 'list'